In [1]:
import sys

sys.path.append('/scratch/gpfs/JHA/mb5157/repos/graphmert/transformers/src')

In [4]:
import os
import sys
import json
import ast
import re
import nest_asyncio, asyncio
import logging
from pathlib import Path
import pyarrow as pa
from enum import Enum, auto

from datasets import Dataset, Features, Sequence, Value, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer
from google import genai
from google.genai import types

/scratch/gpfs/JHA/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
from importlib import reload # reload after updating prompts

import prompts_scores
reload(prompts_scores)
from prompts_scores import system_prompt_fact_score_seq_only, system_prompt_fact_score_general, system_prompt_validity_score

class Prompt(Enum):
    FactScore_seq_only = auto()
    FactScore_llm_knowledge_and_seq = auto()
    ValidityScore = auto()

PROMPTS = {
    Prompt.FactScore_seq_only: system_prompt_fact_score_seq_only,
    Prompt.FactScore_llm_knowledge_and_seq: system_prompt_fact_score_general,
    Prompt.ValidityScore: system_prompt_validity_score
}

# set here which score to run
PROMPT_MODE = Prompt.ValidityScore
SYSTEM_CONTEXT = PROMPTS[PROMPT_MODE]

In [27]:
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY env var not set – please `export GEMINI_API_KEY=…` before launching.")
    
MODEL = "gemini-2.0-flash"
model_name = "gemini-2.0-flash"

# MODEL = "gemini-2.5-flash"
# model_name = "gemini-2.5-flash"

tokenizer = AutoTokenizer.from_pretrained(
    "/scratch/gpfs/JHA/mb5157/tokenizers/biomedbert_fast_tokenizer"
)

SAVE_CHUNK      = 1000  # save result each save every BATCH_SIZE examples
PRINTOUT        = True  # print progress
TAKE_SUBSET     = False # run only on subset
SUBSET_SIZE      = 100 # size for test run
BATCH_SIZE      = 128   # how many concurrent Gemini calls|

# TIME_SLEEP varies across the models
TIME_SLEEP = 0.05
# TIME_SLEEP = 1

PRINT_RAW_RESPONSE = False # set True for test

In [28]:
logging.basicConfig(level=logging.INFO, force=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
handler.setFormatter(fmt)
logger.addHandler(handler)

# Quiet down other noisy loggers
logging.getLogger("google_genai.models").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [29]:
def extract_answer_fact_score(response: str) -> str:
    """
    Extracts the rightmost JSON list from an LLM response string
    """
    start, end = response.rfind('['), response.rfind(']')
    if start == -1 or end == -1 or start > end:
        return ""
    candidate = response[start+1:end].strip().lower()
    if candidate in ("yes", "no") :
        return candidate
    else:
        return ""

def extract_answer_validty_score(response: str) -> str:
    """
    Extracts the rightmost JSON list from an LLM response string
    """
    start, end = response.rfind('['), response.rfind(']')
    if start == -1 or end == -1 or start > end:
        return ""
    candidate = response[start+1:end].strip().lower()
    if candidate in ("yes", "no", "maybe") :
        return candidate
    else:
        return ""

if PROMPT_MODE == Prompt.ValidityScore:
    extract_answer = extract_answer_validty_score
if PROMPT_MODE == Prompt.FactScore_llm_knowledge_and_seq or PROMPT_MODE == Prompt.FactScore_seq_only:
    extract_answer = extract_answer_fact_score

In [30]:
async def gemini_call(client, user_query: str, count_only, max_retries: int=3) -> str:
    """
    Wrap Gemini call in a retrying try/except so it never blows up.
    count_only: mock call for counting input tokens.
    """
    global input_tokens, output_tokens
    # count_tokens is mock request, generate_content is actual paid request
    if count_only:
        call_fn = client.aio.models.count_tokens
        config = None
        contents = [
            {"role": "user", "parts": [{"text": SYSTEM_CONTEXT}]},
            {"role": "user", "parts": [{"text": user_query}]},
        ]
    else:
        call_fn = client.aio.models.generate_content
        config = types.GenerateContentConfig(system_instruction=SYSTEM_CONTEXT)
        contents = [{
                "role": "user",
                "parts": [{"text": user_query}]
            }
        ]
        
    for attempt in range(1, max_retries + 1):
        try:            
            resp = await call_fn(model=MODEL, contents=contents, config=config)
            if count_only:
                # count_tokens returns an int of prompt tokens
                input_tokens += resp.total_tokens
                return ""

            # real response path
            metadata = resp.usage_metadata
            pt = metadata.prompt_token_count or 0
            ct = metadata.candidates_token_count or 0
            input_tokens += pt
            output_tokens += ct
            return getattr(resp, "text", "") or ""

        except Exception as e:
            # catch HTTPError, ValidationError, ServerError, etc.
            logger.warning(f"Gemini call attempt {attempt}/{max_retries} failed: {e!r}")
            if attempt < max_retries:
                await asyncio.sleep(0.5 * attempt)    # simple backoff
            else:
                logger.error(f"All {max_retries} Gemini attempts failed for query: {user_query[:50]!r}")
                return ""  # give up


async def process_batch(client, slice_of_examples, count_only, print_raw_response=PRINT_RAW_RESPONSE):
    """
    Fire off up to BATCH_SIZE concurrent requests.
    """
    tasks = []
    for example in slice_of_examples:
        head = example["head"]
        relation = example["relation"]
        tail = example["tail"]
        if PROMPT_MODE != Prompt.ValidityScore:
            sequence = example["text"]
            uq = f"Input:\n{sequence}\n\nhead: {head}\nrelation: {relation}\ntail: {tail}\n\nOutput:"
        else:
            uq = f"{head}\nrelation: {relation}\ntail: {tail}\n\nOutput:"
    

        tasks.append(gemini_call(client, uq, count_only))
        await asyncio.sleep(TIME_SLEEP)
        
    raw_resps = await asyncio.gather(*tasks)

    if print_raw_response:
        print("RAW RESPONSE")
        for raw_resp in raw_resps:
            print(raw_resp)

    
    cleaned = [extract_answer(r) for r in raw_resps]
    return cleaned
    

async def run_all(client, dataset: Dataset, chunk_start: int, chunk_end: int, count_only=True):
    """
    Split into batches
    """
    n = len(dataset)
    all_resp = [None] * n

    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        slice_ds = dataset.select(range(start, end))
        cleaned  = await process_batch(client, slice_ds, count_only)
        all_resp[start:end] = cleaned
        if PRINTOUT:
            logger.info(f"Gemini: processed {end}/{n}")
            
    return all_resp

In [31]:
def get_chunk_suffix(chunk_start, chunk_end):    
    if PROMPT_MODE == Prompt.ValidityScore:
        prefix = 'validated'
    elif PROMPT_MODE == Prompt.FactScore_seq_only:
        prefix = 'accepted_seq_only'
    elif PROMPT_MODE == Prompt.FactScore_llm_knowledge_and_seq:
        prefix = 'accepted'
        
    if TAKE_SUBSET:
        save_name = f"{prefix}_{model_name}_subset_{SUBSET_SIZE}_{chunk_start}-{chunk_end}"
    else:
        save_name = f"{prefix}_{model_name}_{chunk_start}-{chunk_end}"
    return save_name


def load_dataset_with_triples(predictions_path):
    if predictions_path.endswith('.csv'):
        ds_full = Dataset.from_csv(predictions_path)
    else:
        ds_full = Dataset.load_from_disk(predictions_path)
    if 'relation_type' in ds_full.column_names:
        ds_full = ds_full.rename_column('relation_type', 'relation')
    if 'sequence' in ds_full.column_names:
        ds_full = ds_full.rename_column('sequence', 'text')

    return ds_full

In [32]:
async def main(predictions_path: str, path_to_save, count_only=True, start_idx: int = 0, end_idx=None):
    ds_full = load_dataset_with_triples(predictions_path)
    if TAKE_SUBSET:
        overall_end = min(start_idx + SUBSET_SIZE, len(ds_full))
    else:
        overall_end = len(ds_full) if end_idx is None else min(len(ds_full), end_idx)

    client = genai.Client(api_key=GEMINI_API_KEY)
          
    for chunk_start in range(start_idx, overall_end, SAVE_CHUNK):
        chunk_end = min(chunk_start + SAVE_CHUNK, overall_end)
        ds_chunk = ds_full.select(range(chunk_start, chunk_end))
        entities = await run_all(client, ds_chunk, chunk_start, chunk_end, count_only=count_only)
        
        assert len(entities) == ds_chunk.num_rows, f"{len(entities)=} vs {ds_chunk.num_rows=}"
        if not count_only:
            # coerce any stray string into a single‐element list
            fixed = [[str(x) for x in t] if isinstance(t, list) else [str(t)] for t in entities]
            entities_arr = pa.array(fixed, type=pa.list_(pa.string()))
            
            ds_chunk = ds_chunk.add_column("response", fixed)
            save_name = get_chunk_suffix(chunk_start, chunk_end)
            out_dir = os.path.join(str(path_to_save), save_name)
            ds_chunk.save_to_disk(out_dir)
            logger.info(f"Saved chunk {chunk_start}-{chunk_end} to {out_dir}")


In [33]:
input_tokens = 0
output_tokens = 0

count_only = False # True means no real requests

# csv or dataset
predictions_path = '/scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/triples_with_chunks_sampled.csv'

path_to_save = os.path.dirname(predictions_path)
start_idx = 0

logger.info(f'request with {model_name}')
await main(predictions_path, path_to_save, count_only=count_only, start_idx=start_idx, end_idx=None)

print('===finished===')

2025-12-15 11:22:28,627 INFO request with gemini-2.0-flash
2025-12-15 11:22:28,627 INFO request with gemini-2.0-flash


INFO:__main__:request with gemini-2.0-flash


2025-12-15 11:22:36,916 INFO Gemini: processed 128/1000
2025-12-15 11:22:36,916 INFO Gemini: processed 128/1000


INFO:__main__:Gemini: processed 128/1000


2025-12-15 11:22:45,032 INFO Gemini: processed 256/1000
2025-12-15 11:22:45,032 INFO Gemini: processed 256/1000


...

2025-12-15 13:09:23,550 INFO Gemini: processed 1000/1000
2025-12-15 13:09:23,550 INFO Gemini: processed 1000/1000


INFO:__main__:Gemini: processed 1000/1000
Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 54059.37 examples/s]

2025-12-15 13:09:23,587 INFO Saved chunk 99000-100000 to /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_gemini-2.0-flash_99000-100000
2025-12-15 13:09:23,587 INFO Saved chunk 99000-100000 to /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_gemini-2.0-flash_99000-100000



INFO:__main__:Saved chunk 99000-100000 to /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_gemini-2.0-flash_99000-100000


===finished===


In [34]:
print('input_tokens:', input_tokens)
print('output_tokens:', output_tokens)

input_tokens: 5203369
output_tokens: 2312982


In [35]:
print(input_tokens / 10**6 *0.1)
print(output_tokens / 10**6 * 0.4)


0.5203369000000001
0.9251928


## Unite chunks

In [36]:
datasets = []

dataset_path = os.path.dirname(predictions_path)
dataset_len = len(load_dataset_with_triples(predictions_path))
for i in range(0, dataset_len, 1000):
    last = dataset_len if i + 1000 > dataset_len else i + 1000
    suffix = get_chunk_suffix(i, last)
    path = os.path.join(dataset_path, suffix)
    datasets.append(load_from_disk(path))

united_dataset = concatenate_datasets(datasets)
print(len(united_dataset))
assert len(united_dataset) == dataset_len

100000


In [37]:
suffix = get_chunk_suffix(i, last)
suffix = '_'.join(suffix.split('_')[:-1]) + '_all'
output_path = os.path.join(dataset_path, suffix)
united_dataset.save_to_disk(output_path)
print(f'saved to {output_path}')

Saving the dataset (2/2 shards): 100%|██████████| 100000/100000 [00:02<00:00, 47762.72 examples/s]

saved to /scratch/gpfs/JHA/mb5157/large_data/graphrag_results/qwen14b_fixed_rel_kg/validated_gemini-2.0-flash_all


In [38]:
import shutil

dataset_path = os.path.dirname(predictions_path)
for i in range(0, dataset_len, 1000):
    last = dataset_len if i + 1000 > dataset_len else i + 1000
    suffix = get_chunk_suffix(i, last)
    path = os.path.join(dataset_path, suffix)
    shutil.rmtree(path)